In [94]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from scipy.stats import zscore

warnings.filterwarnings("ignore")

In [95]:
data = pd.read_excel('Telco-Customer-Churn.xlsx');
data

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.50,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.90,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.60,Yes


```
1. Realizar un análisis exploratorio de los datos para comprender las características y la
distribución de los datos.

```

In [96]:
data.head()


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [97]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [98]:
data.describe()

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
count,7043.000000,7043.000000,7043.000000,7032.000000
mean,0.162147,32.371149,64.761692,2283.300441
std,0.368612,24.559481,30.090047,2266.771362
min,0.000000,0.000000,18.250000,18.800000
25%,0.000000,9.000000,35.500000,401.450000
50%,0.000000,29.000000,70.350000,1397.475000
75%,0.000000,55.000000,89.850000,3794.737500
max,1.000000,72.000000,118.750000,8684.800000


In [99]:
data['gender'].value_counts()


gender
Male      3555
Female    3488
Name: count, dtype: int64

```
 2. Preprocesar los datos, incluyendo la limpieza, la transformación y la normalización
de las características según sea necesario.
```

In [100]:
# Crear el nuevo DataFrame con las variables relevantes y la variable objetivo
df_relevante = data[
    [
        "tenure",
        "Contract",
        "MonthlyCharges",
        "TotalCharges",
        "InternetService",
        "OnlineSecurity",
        "TechSupport",
        "Churn",
    ]
]

# Mostrar las primeras filas del nuevo DataFrame
df_relevante.head(10)

,tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,Churn
0,1,Month-to-month,29.85,29.85,DSL,No,No,No
1,34,One year,56.95,1889.50,DSL,Yes,No,No
2,2,Month-to-month,53.85,108.15,DSL,Yes,No,Yes
3,45,One year,42.30,1840.75,DSL,Yes,Yes,No
4,2,Month-to-month,70.70,151.65,Fiber optic,No,No,Yes
5,8,Month-to-month,99.65,820.50,Fiber optic,No,No,Yes
6,22,Month-to-month,89.10,1949.40,Fiber optic,No,No,No
7,10,Month-to-month,29.75,301.90,DSL,Yes,No,No
8,28,Month-to-month,104.80,3046.05,Fiber optic,No,Yes,Yes
9,62,One year,56.15,3487.95,DSL,Yes,No,No


In [101]:
# Convertir variables binarias en 0 y 1 para las columnas "OnlineSecurity" y "TechSupport"
df_relevante.replace({"No": 0, "Yes": 1}, inplace=True)
# Convertir variables binarias en 0 y 1 para la columna "Contract"
df_relevante.replace({"Month-to-month": 0, "One year": 1, "Two year": 2}, inplace=True)
df_relevante.replace({"DSL": 0, "Fiber optic": 1,"No internet service":2}, inplace=True)
df_relevante

,tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,Churn
0,1,0,29.85,29.85,0,0,0,0
1,34,1,56.95,1889.50,0,1,0,0
2,2,0,53.85,108.15,0,1,0,1
3,45,1,42.30,1840.75,0,1,1,0
4,2,0,70.70,151.65,1,0,0,1
...,...,...,...,...,...,...,...,...
7038,24,1,84.80,1990.50,0,1,1,0
7039,72,1,103.20,7362.90,1,0,0,0
7040,11,0,29.60,346.45,0,1,0,0
7041,4,0,74.40,306.60,1,0,0,1


In [102]:
df_relevante=df_relevante.drop_duplicates()
df_relevante

,tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,Churn
0,1,0,29.85,29.85,0,0,0,0
1,34,1,56.95,1889.50,0,1,0,0
2,2,0,53.85,108.15,0,1,0,1
3,45,1,42.30,1840.75,0,1,1,0
4,2,0,70.70,151.65,1,0,0,1
...,...,...,...,...,...,...,...,...
7038,24,1,84.80,1990.50,0,1,1,0
7039,72,1,103.20,7362.90,1,0,0,0
7040,11,0,29.60,346.45,0,1,0,0
7041,4,0,74.40,306.60,1,0,0,1


In [103]:
df_relevante = df_relevante.dropna();
df_relevante

,tenure,Contract,MonthlyCharges,TotalCharges,InternetService,OnlineSecurity,TechSupport,Churn
0,1,0,29.85,29.85,0,0,0,0
1,34,1,56.95,1889.50,0,1,0,0
2,2,0,53.85,108.15,0,1,0,1
3,45,1,42.30,1840.75,0,1,1,0
4,2,0,70.70,151.65,1,0,0,1
...,...,...,...,...,...,...,...,...
7038,24,1,84.80,1990.50,0,1,1,0
7039,72,1,103.20,7362.90,1,0,0,0
7040,11,0,29.60,346.45,0,1,0,0
7041,4,0,74.40,306.60,1,0,0,1


```
3. Dividir los datos en conjuntos de entrenamiento y prueba.

```

In [104]:
# Separar los datos en características (X) y variable objetivo (y)
X = df_relevante.drop("Churn", axis=1)
y = df_relevante["Churn"]

In [105]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [106]:
# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

```4. Entrenar al menos 3 algoritmos y optimizar sus hiperparametros```

In [107]:
# Definir los parámetros a ajustar
param_grid = {
    "n_neighbors": [1, 3, 5, 7, 9],
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan"],
}

# Crear un clasificador KNN
knn = KNeighborsClassifier()

# Configurar la búsqueda de hiperparámetros utilizando validación cruzada
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring="accuracy")

# Realizar la búsqueda de hiperparámetros en los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Obtener el mejor modelo encontrado
best_knn = grid_search.best_estimator_

# Evaluar el rendimiento del mejor modelo en el conjunto de prueba
test_accuracy = best_knn.score(X_test_scaled, y_test)

print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)
print(
    f"Precisión en el conjunto de prueba con los mejores hiperparámetros: {test_accuracy}"
)

# Predecir las etiquetas del conjunto de prueba utilizando el mejor modelo
y_pred = best_knn.predict(X_test_scaled)

# Generar un informe de clasificación con las métricas pertinentes
classification_rep = classification_report(y_test, y_pred)

print("Informe de clasificación:")
print(classification_rep)

Mejores hiperparámetros encontrados:
{'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
Precisión en el conjunto de prueba con los mejores hiperparámetros: 0.7873270211216314
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.89      0.86      1037
           1       0.58      0.47      0.52       336

    accuracy                           0.79      1373
   macro avg       0.71      0.68      0.69      1373
weighted avg       0.78      0.79      0.78      1373



In [108]:
# Definir los hiperparámetros a ajustar
param_grid = {
    "max_depth": [3, 5, 7, 10],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

# Inicializar el clasificador de árbol de decisión
dt_classifier = DecisionTreeClassifier(random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(dt_classifier, param_grid, cv=5, scoring="accuracy")

# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params)

# Obtener la mejor puntuación de precisión
best_score = grid_search.best_score_
print(f"Mejor precisión encontrada: {best_score}")

# Evaluar el rendimiento del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test_scaled, y_test)
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

# Predecir las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test_scaled)

# Generar un informe de clasificación con las métricas pertinentes
classification_rep = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(classification_rep)

Mejores hiperparámetros encontrados:
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10}
Mejor precisión encontrada: 0.7879402842327062
Precisión en el conjunto de prueba: 0.7909686817188638
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.89      0.87      1037
           1       0.59      0.48      0.53       336

    accuracy                           0.79      1373
   macro avg       0.72      0.68      0.70      1373
weighted avg       0.78      0.79      0.78      1373



In [109]:
# Definir los hiperparámetros a ajustar
param_grid = {
    "C": [0.001, 0.01, 0.1, 1, 10, 100],
    "penalty": ["l1", "l2"],
    "solver": ["liblinear", "saga"],
}

# Inicializar el clasificador de Regresión Logística
logistic_classifier = LogisticRegression(random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(logistic_classifier, param_grid, cv=5, scoring="accuracy")

# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params)

# Obtener la mejor puntuación de precisión
best_score = grid_search.best_score_
print(f"Mejor precisión encontrada: {best_score}")

# Evaluar el rendimiento del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test_scaled, y_test)
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

# Predecir las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test_scaled)

# Generar un informe de clasificación con las métricas pertinentes
classification_rep = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(classification_rep)

Mejores hiperparámetros encontrados:
{'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Mejor precisión encontrada: 0.7903103845061793
Precisión en el conjunto de prueba: 0.8055353241077932
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.84      0.91      0.88      1037
           1       0.64      0.48      0.55       336

    accuracy                           0.81      1373
   macro avg       0.74      0.69      0.71      1373
weighted avg       0.79      0.81      0.80      1373



In [110]:
# Definir los hiperparámetros a ajustar
param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, 20],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
}

# Inicializar el clasificador de Random Forest
rf_classifier = RandomForestClassifier(random_state=42)

# Inicializar GridSearchCV
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring="accuracy")

# Ajustar GridSearchCV con los datos de entrenamiento
grid_search.fit(X_train_scaled, y_train)

# Obtener los mejores hiperparámetros
best_params = grid_search.best_params_
print("Mejores hiperparámetros encontrados:")
print(best_params)

# Obtener la mejor puntuación de precisión
best_score = grid_search.best_score_
print(f"Mejor precisión encontrada: {best_score}")

# Evaluar el rendimiento del mejor modelo en el conjunto de prueba
best_model = grid_search.best_estimator_
test_accuracy = best_model.score(X_test_scaled, y_test)
print(f"Precisión en el conjunto de prueba: {test_accuracy}")

# Predecir las etiquetas del conjunto de prueba
y_pred = best_model.predict(X_test_scaled)

# Generar un informe de clasificación con las métricas pertinentes
classification_rep = classification_report(y_test, y_pred)
print("Informe de clasificación:")
print(classification_rep)

Mejores hiperparámetros encontrados:
{'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
Mejor precisión encontrada: 0.7959560184839262
Precisión en el conjunto de prueba: 0.8033503277494537
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.83      0.92      0.88      1037
           1       0.64      0.44      0.52       336

    accuracy                           0.80      1373
   macro avg       0.74      0.68      0.70      1373
weighted avg       0.79      0.80      0.79      1373



```
5. Evaluar el rendimiento de los modelos utilizando métricas de evaluación pertinentes,
como precisión, sensibilidad, valor F1, entre otras.
```

### Evaluación de todos los modelos:

### KNN:
- Mejores hiperparámetros: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
- Precisión en el conjunto de prueba: 0.787
- F1-score para la clase 1: 0.52

### Árbol de decisión:
- Mejores hiperparámetros: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 10}
- Precisión en el conjunto de prueba: 0.791
- F1-score para la clase 1: 0.53

### Regresión Logística:
- Mejores hiperparámetros: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
- Precisión en el conjunto de prueba: 0.806
- F1-score para la clase 1: 0.55

### Random Forest:
- Mejores hiperparámetros: {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
- Precisión en el conjunto de prueba: 0.803
- F1-score para la clase 1: 0.52

### Evaluación general:
- En términos de precisión en el conjunto de prueba, el mejor desempeño se observa con la regresión logística.
- En cuanto al F1-score para la clase 1, la regresión logística también muestra el mejor rendimiento.
- Sin embargo, todos los modelos muestran resultados bastante similares en general, con diferencias menores entre ellos.

En resumen, la regresión logística parece ser la opción preferida en este caso basándonos en las métricas de precisión y F1-score.